## Transfer Learning VGG 16 and VGG 19 using Keras

In [5]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [7]:
# re-size all the images to this
IMAGE_SIZE = [224,224]

train_path = 'Datasets/train'
valid_path = 'Datasets/test'

In [8]:
# Import the Vgg 16 library as shown below and add preprocessing Layer to the front of VGG
# Here we will use imagenet weights
resnet = ResNet50(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False) 

94765736/94765736 [==============================] - 78s 1us/step


In [9]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [10]:
# useful for getting number of output classes
folders = glob('Datasets/train/')

In [11]:
x = Flatten()(resnet.output)

In [12]:
prediction = Dense(len(folders), activation='softmax')(x)

# create model object
model = Model(inputs=resnet.input, outputs = prediction)

In [13]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [14]:
# tell the model what cost and optimization method to use
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [15]:
# use the image data generator to import the images from the dataset

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
# Make sure you provide the same target size as initialized for the image size
training_set = train_datagen.flow_from_directory('Datasets/train',
                                                  target_size =(224,224),
                                                  batch_size = 32,
                                                  class_mode = 'categorical')

Found 64 images belonging to 3 classes.


In [20]:
test_set = test_datagen.flow_from_directory('Datasets/test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 58 images belonging to 3 classes.


In [22]:
# fit the model
r = model.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 50,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

C:\conda_temp\ipykernel_15424\284548119.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'Adam/Adam/update_1/ResourceApplyAdam' defined at (most recent call last):
    File "d:\car-price-prediction\venv\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\car-price-prediction\venv\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "d:\car-price-prediction\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\car-price-prediction\venv\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "d:\car-price-prediction\venv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "d:\car-price-prediction\venv\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "d:\car-price-prediction\venv\lib\asyncio\base_events.py", line 600

In [ ]:
# plot the loss
plt.plot(r.history)